## Installations :

In [ ]:
!pip install googletrans
!pip install SpeechRecognition
!pip install pipwin
!pipwin install pyaudio
!pip install pyttsx3
!pip install pydub

## Code :

In [24]:
from googletrans import Translator
import sounddevice as sd
from scipy.io.wavfile import write
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence
import speech_recognition as sr


In [25]:
fs = 44100  # Sample rate
seconds = 5  # Duration of recording

myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
sd.wait()  # Wait until recording is finished
write('output.wav', fs, myrecording)

In [7]:
r = sr.Recognizer()

In [8]:
# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """

    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [26]:
path = "output.wav"
result=get_large_audio_transcription(path)
print("\nFull text:", get_large_audio_transcription(path))
translator=Translator()
test=translator.translate(result,src='ur',des='en')
test.text

audio-chunks\chunk1.wav : Kya kar rahe ho. 
audio-chunks\chunk1.wav : Kya kar rahe ho. 

Full text: Kya kar rahe ho. 


'What are you doing.'

## Covert MP3 to .WAV

In [ ]:
from os import path
from pydub import AudioSegment

# files                                                                         
src = "mp3_file_name.mp3"
dst = "mp3_converted_to_wav.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")